In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 24.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip tales_archive.zip

Archive:  tales_archive.zip
  inflating: tales.csv               


In [ ]:
import pandas as pd

In [ ]:
df_rec = pd.read_csv('/content/tales.csv', sep='\t')

In [ ]:
df_rec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28055 entries, 0 to 28054
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Tale,Label  28055 non-null  object
dtypes: object(1)
memory usage: 219.3+ KB


In [ ]:
print(df_rec.loc[12, 'Tale,Label'])

Откуда снежинки? Кто вяжет снежинки и с неба бросает? Узоров таких даже мама не знает И пряжи нигде не встречали такой - Наш город весь белый, слегка голубой. А, может быть, их вырезают из ситца И тихо спускают ко мне на ресницы? Слезой на щеках они медленно тают... Откуда снежинки? Хоть кто-нибудь знает?  


In [ ]:
for i in df_rec.index:
  df_rec.loc[i, 'Tale,Label'] = df_rec.loc[i, 'Tale,Label'][:-3]

In [ ]:
print(df_rec.loc[22, 'Tale,Label'])

Я живу у Дедушки Мороза Я живу у Дедушки, Дедушки Мороза Личико румяное и щечки словно розы.  Мне косички белые вьюга заплетала, Красивые узоры на шубке рисовала. Тетушка Метелица связала руковички Я девочка снегурочка, самой зимы сестричка. 


In [ ]:
data = df_rec.loc[:28000, 'Tale,Label']

In [ ]:
import re
from sklearn.model_selection import train_test_split

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [ ]:
train, test = train_test_split(data, test_size=0.2)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')


In [ ]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 22400
Test dataset length: 5601


In [ ]:
from transformers import AutoTokenizer
#sberbank-ai/rugpt3large_based_on_gpt2
#sberbank-ai/rugpt3medium_based_on_gpt2
#sberbank-ai/rugpt3small_based_on_gpt2

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=256)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=256)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (2362136 > 2048). Running this sequence through the model will result in indexing errors


In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-chief", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=12, # batch size for training
    per_device_eval_batch_size=18,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=5000, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

Step,Training Loss
500,3.789000
1000,3.642900
1500,3.556300
2000,3.379700
2500,3.317500
3000,3.238600
3500,3.126500
4000,3.091000
4500,3.022400
5000,2.951600


TrainOutput(global_step=7690, training_loss=3.1479392578759957, metrics={'train_runtime': 6866.9736, 'train_samples_per_second': 13.437, 'train_steps_per_second': 1.12, 'total_flos': 1.205470789632e+16, 'train_loss': 3.1479392578759957, 'epoch': 10.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/gpt2small_based_tales_model/')

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/gpt2small_based_tales_model/gpt_chf')

('/content/drive/MyDrive/gpt2small_based_tales_model/gpt_chf/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2small_based_tales_model/gpt_chf/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2small_based_tales_model/gpt_chf/vocab.json',
 '/content/drive/MyDrive/gpt2small_based_tales_model/gpt_chf/merges.txt',
 '/content/drive/MyDrive/gpt2small_based_tales_model/gpt_chf/added_tokens.json',
 '/content/drive/MyDrive/gpt2small_based_tales_model/gpt_chf/tokenizer.json')

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/gpt2small_based_tales_model/gpt_chf')
model.save_pretrained('/content/drive/MyDrive/gpt2small_based_tales_model/model_gpt_chf')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/gpt2small_based_tales_model/gpt_chf")
model1 = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/gpt2small_based_tales_model/model_gpt_chf")

In [ ]:
prefix = "Жили были старик со старухою"

In [ ]:
tokens = tokenizer(prefix, return_tensors='pt')
#tokens = {k: v.to(model.device) for k, v in tokens.items()}

In [ ]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens,
    #end_token=end_token_id,
    do_sample=False,
    max_length=size+150,
    repetition_penalty=5.,
    temperature=0.2,
    num_beams=15,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Жили были старик со старухою, и так случилось, что жил он в одной комнате с ними. Жили они бедно, потому что не было у них ни денег, ни прописки; но зато была какая-то необыкновенная сила, которая заставляла их каждый день ходить на рынок за чем-нибудь вкусненьким: то хлеб покупали, то пирожки пекли, то козацкие лепешки варили – словом, делали все, что нужно для приятного времяпрепровождения.  Как же меня бесят люди, которые вместо того, чтобы сказать "я тебя люблю", начинают говорить "я тоже тебя люблю" или "ты мне нравишься". И это при том, что я сама по себе довольно симпатичная девушка (хотя выгляжу далеко не красоткой),
